In [1]:
# # header
import sys
sys.path.append(r"../")

%load_ext autoreload
%autoreload 2

In [2]:
# # built-in modules
import os
import argparse
from pprint import pformat
from collections import OrderedDict
import random
# # Torch modules
import torch
from torchvision import transforms, datasets
# # internal imports
from prelude import startup_folders, get_device, load_dicts
from src.composer import CelebACrop, CelebGender
from src.model import AttentionModel
from src.utils import plot_all
from src.utils import build_loaders
from src.conductor import AttentionTrain

In [3]:
start_folder = r"../pretrained/celeba"
results_folder, logger = startup_folders(start_folder, name=f"exp_celeb")
data_path = r"../data"

../pretrained/celeba/1728906947 was created!


In [4]:
model_params = load_dicts(start_folder, "model_params")
tsk = load_dicts(start_folder, "tasks")
train_params = load_dicts(start_folder, "train_params")
n_iter = tsk["Celeb"]['params']['n_iter']
DeVice, num_workers, pin_memory = get_device()
print(f"model_params: {model_params}")
print(f"tasks: {tsk}")
print(f"train_params: {train_params}")

Device set to mps
model_params: {'in_dims': [3, 128, 128], 'n_classes': 2, 'out_dim': 2, 'normalize': True, 'softness': 0.5, 'channels': [3, 4, 8, 8, 16, 16, 32, 32], 'residuals': False, 'kernels': 3, 'strides': 1, 'paddings': 1, 'conv_bias': True, 'conv_norms': None, 'conv_dropouts': 0.0, 'conv_funs': ReLU(), 'deconv_funs': Tanh(), 'deconv_norms': None, 'pools': 2, 'rnn_dims': [32, 8], 'rnn_bias': True, 'rnn_dropouts': 0.0, 'rnn_funs': ReLU(), 'n_tasks': 1, 'task_weight': False, 'task_bias': False, 'task_funs': None, 'rnn_to_fc': True, 'trans_fun': Identity(), 'norm_mean': [0.5, 0.5, 0.5], 'norm_std': [1.0, 1.0, 1.0]}
tasks: {'Celeb': {'composer': 'CelebACrop', 'key': 0, 'params': {'n_iter': 2, 'hair_dir': None, 'in_dims': [3, 128, 128], 'padding': 0, 'noise': 0.25, 'which': 0}, 'datasets': ['CelebACrop', 'CelebACrop', 'CelebACrop'], 'dataloaders': [None, None, None], 'loss_w': [0.0, 0.0, 1.0], 'loss_s': [0, None], 'has_prompt': False}}
train_params: {'n_epochs': 32, 'batch_size': 128

In [5]:
# using "which" parameter in the class to select the hair color and gender 
tasks = OrderedDict({})
tasks["Celeb_all"] = {
    "composer": CelebACrop,  # composer (torch Dataset)
    "key": 0,  # key for the task
    "params": {"n_iter": n_iter, "hair_dir": data_path, "in_dims": model_params["in_dims"], "padding": 0, "noise": 0.25, "which": 0},
    "datasets": [],
    "dataloaders": [],
    "loss_w": (0.0, 0.0, 1.0),  # Loss weights (Cross-Entropy (CE), MSE for attention, CE last label)
    "loss_s": (None, None),  # Loss slices (CE, MSE for attention)
    "has_prompt": False,  # has prompt or not (only used for top-down Search)
}
tasks["Celeb_fblonde"] = {
    "composer": CelebACrop,  # composer (torch Dataset)
    "key": 0,  # key for the task
    "params": {"n_iter": n_iter, "hair_dir": data_path, "in_dims": model_params["in_dims"], "padding": 0, "noise": 0.25, "which": 1},
    "datasets": [],
    "dataloaders": [],
    "loss_w": (0.0, 0.0, 1.0),  # Loss weights (Cross-Entropy (CE), MSE for attention, CE last label)
    "loss_s": (None, None),  # Loss slices (CE, MSE for attention)
    "has_prompt": False,  # has prompt or not (only used for top-down Search)
}
tasks["Celeb_fblack"] = {
    "composer": CelebACrop,  # composer (torch Dataset)
    "key": 0,  # key for the task
    "params": {"n_iter": n_iter, "hair_dir": data_path, "in_dims": model_params["in_dims"], "padding": 0, "noise": 0.25, "which": 2},
    "datasets": [],
    "dataloaders": [],
    "loss_w": (0.0, 0.0, 1.0),  # Loss weights (Cross-Entropy (CE), MSE for attention, CE last label)
    "loss_s": (None, None),  # Loss slices (CE, MSE for attention)
    "has_prompt": False,  # has prompt or not (only used for top-down Search)
}
tasks["Celeb_mblonde"] = {
    "composer": CelebACrop,  # composer (torch Dataset)
    "key": 0,  # key for the task
    "params": {"n_iter": n_iter, "hair_dir": data_path, "in_dims": model_params["in_dims"], "padding": 0, "noise": 0.25, "which": 3},
    "datasets": [],
    "dataloaders": [],
    "loss_w": (0.0, 0.0, 1.0),  # Loss weights (Cross-Entropy (CE), MSE for attention, CE last label)
    "loss_s": (None, None),  # Loss slices (CE, MSE for attention)
    "has_prompt": False,  # has prompt or not (only used for top-down Search)
}
tasks["Celeb_mblack"] = {
    "composer": CelebACrop,  # composer (torch Dataset)
    "key": 0,  # key for the task
    "params": {"n_iter": n_iter, "hair_dir": data_path, "in_dims": model_params["in_dims"], "padding": 0, "noise": 0.25, "which": 4},
    "datasets": [],
    "dataloaders": [],
    "loss_w": (0.0, 0.0, 1.0),  # Loss weights (Cross-Entropy (CE), MSE for attention, CE last label)
    "loss_s": (None, None),  # Loss slices (CE, MSE for attention)
    "has_prompt": False,  # has prompt or not (only used for top-down Search)
}


In [6]:
# datasets and dataloaders
train_ds = datasets.CelebA(root=data_path, split='train', transform=transforms.ToTensor())
valid_ds = datasets.CelebA(root=data_path, split='valid', transform=transforms.ToTensor())
test_ds = datasets.CelebA(root=data_path, split='test', transform=transforms.ToTensor())
DeVice, num_workers, pin_memory = get_device()
for o in tasks:
    # if o == "CelebGender":
    #     tasks[o]["datasets"].append(tasks[o]["composer"](train_ds, **tasks[o]["params"], kind="train"))
    #     tasks[o]["datasets"].append(tasks[o]["composer"](valid_ds, **tasks[o]["params"], kind="valid"))
    #     tasks[o]["datasets"].append(tasks[o]["composer"](test_ds, **tasks[o]["params"], kind="test"))
    # else:
    tasks[o]["datasets"].append(tasks[o]["composer"](train_ds, **tasks[o]["params"], kind="train"))
    tasks[o]["datasets"].append(tasks[o]["composer"](valid_ds, **tasks[o]["params"], kind="valid"))
    tasks[o]["datasets"].append(tasks[o]["composer"](test_ds, **tasks[o]["params"], kind="test"))
    tasks[o]["dataloaders"] = build_loaders(tasks[o]["datasets"], batch_size=train_params["batch_size"], num_workers=num_workers, pin_memory=pin_memory)


Device set to mps
Loading train_hair_ids.pt from file!
Loading valid_hair_ids.pt from file!
Loading test_hair_ids.pt from file!
Loading train_hair_ids.pt from file!
Loading valid_hair_ids.pt from file!
Loading test_hair_ids.pt from file!
Loading train_hair_ids.pt from file!
Loading valid_hair_ids.pt from file!
Loading test_hair_ids.pt from file!
Loading train_hair_ids.pt from file!
Loading valid_hair_ids.pt from file!
Loading test_hair_ids.pt from file!
Loading train_hair_ids.pt from file!
Loading valid_hair_ids.pt from file!
Loading test_hair_ids.pt from file!


In [7]:
# create a blank model
model = AttentionModel(**model_params)
conductor = AttentionTrain(model, None, None, tasks, logger, results_folder)

# load states into the model
model_dir = os.path.join(start_folder, "model" + ".pth")
assert os.path.exists(model_dir), "Could not find the model.pth in the given dir!"
model.load_state_dict(torch.load(model_dir, map_location=DeVice))

<All keys matched successfully>

In [8]:
# plotting...
plot_all(10, model, tasks, results_folder, "_test", DeVice, logger, False, "test")

In [9]:
# evaluating...
conductor.eval(DeVice, "test", False)

testing...


  Task Celeb_all:
    CEi Loss: 5.390714    CEe Loss: 0.163816    Pix Err: 0.000000    Att Acc: 0.000000    Cls Acc: 7526/8082
  Task Celeb_fblonde:
    CEi Loss: 5.113946    CEe Loss: 0.091970    Pix Err: 0.000000    Att Acc: 0.000000    Cls Acc: 2385/2480
  Task Celeb_fblack:
    CEi Loss: 8.126309    CEe Loss: 0.252569    Pix Err: 0.000000    Att Acc: 0.000000    Cls Acc: 2562/2875
  Task Celeb_mblonde:
    CEi Loss: 3.405327    CEe Loss: 0.745578    Pix Err: 0.000000    Att Acc: 0.000000    Cls Acc: 110/180
  Task Celeb_mblack:
    CEi Loss: 2.712632    CEe Loss: 0.092475    Pix Err: 0.000000    Att Acc: 0.000000    Cls Acc: 2469/2547
